In [1]:
import json
tree_data = {
    "node1": {
        #"parent": None,
        "data": "Some data for node1"
    },
    "node2": {
        #"parent": "node1",
        "data": "Some data for node2"
    },
    "node3": {
        "parent": "node1",
        "data": "Some data for node3"
    },
    "node4": {
        "parent": "node3",
        "data": "Some data for node4"
    },
    "node5": {
        "parent": "node3",
        "data": "Some data for node5"
    }
}
print(json.dumps(tree_data, indent=2))

{
  "node1": {
    "data": "Some data for node1"
  },
  "node2": {
    "data": "Some data for node2"
  },
  "node3": {
    "parent": "node1",
    "data": "Some data for node3"
  },
  "node4": {
    "parent": "node3",
    "data": "Some data for node4"
  },
  "node5": {
    "parent": "node3",
    "data": "Some data for node5"
  }
}


In [2]:
import treekit as tk

# load a tree from tree_data
tree = tk.FlatTree(tree_data)
print(json.dumps(tree, indent=2))


{
  "node1": {
    "data": "Some data for node1"
  },
  "node2": {
    "data": "Some data for node2"
  },
  "node3": {
    "parent": "node1",
    "data": "Some data for node3"
  },
  "node4": {
    "parent": "node3",
    "data": "Some data for node4"
  },
  "node5": {
    "parent": "node3",
    "data": "Some data for node5"
  }
}


In [3]:
print(tree["node3"])
node3 = tree.get_node("node3")
print(node3)
print(node3['parent'])
print(node3.children())

{'parent': 'node1', 'data': 'Some data for node3'}
ProxyNode(node3: {'parent': 'node1', 'data': 'Some data for node3'})
node1
[ProxyNode(node4: {'parent': 'node3', 'data': 'Some data for node4'}), ProxyNode(node5: {'parent': 'node3', 'data': 'Some data for node5'})]


In [4]:
print(tree.get_root())
for child in tree.get_root().children():
    print(child)

ProxyNode(key=__logical_root__)
ProxyNode(node1: {'data': 'Some data for node1'})
ProxyNode(node2: {'data': 'Some data for node2'})


We show that it's easy to regenerate any JSON files that may have been used
to generate the FlatTree 'tree'. So, JSON is a good format for storing and
transmitting trees. And, of course, `FlatTree` *is* a dictionary. Of course,
if we store an object that has no serializable representation, it cannot be
stored in JSON.

In [5]:
print(json.dumps(tree,  indent=2) == json.dumps(tree_data, indent=2))

True


In [6]:

# let's create a tree from a dict that cannot be serialized to json
non_serializable_tree_data = {
    "node1": {
        #"parent": None,
        # data is a function that cannot be serialized to json
        "data": lambda x: 2*x**3 + 3*x**2 + 4*x + 5
    }
}

non_serializable_tree = tk.FlatTree(non_serializable_tree_data)
print(non_serializable_tree)
print(non_serializable_tree.get_root())

try:
    json.dumps(non_serializable_tree, indent=2)
except TypeError as e:
    print(e)

{'node1': {'data': <function <lambda> at 0x7f3c5b2bbd80>}}
ProxyNode(node1: {'data': <function <lambda> at 0x7f3c5b2bbd80>})
Object of type function is not JSON serializable


In [7]:
child = tree.get_node("node1").add_child(key="node36", data="Some data for node6")
print(json.dumps(tree, indent=2))
print(child)
#print(tree.get_root())

{
  "node1": {
    "data": "Some data for node1"
  },
  "node2": {
    "data": "Some data for node2"
  },
  "node3": {
    "parent": "node1",
    "data": "Some data for node3"
  },
  "node4": {
    "parent": "node3",
    "data": "Some data for node4"
  },
  "node5": {
    "parent": "node3",
    "data": "Some data for node5"
  },
  "node36": {
    "data": "Some data for node6",
    "parent": "node1"
  }
}
ProxyNode(node36: {'data': 'Some data for node6', 'parent': 'node1'})


If we try too add a non-unique node key to the tree, we will get a `KeyError`.

In [8]:
try:
    child2 = tree.get_node("node1").add_child(key="node2", data="Some data for node6!!!")
except KeyError as e:
    print(e)

child7 = child.add_child(key="node7", data="Some data for node7")
print(json.dumps(tree, indent=2))
print(child7)

'Node key already exists'
{
  "node1": {
    "data": "Some data for node1"
  },
  "node2": {
    "data": "Some data for node2"
  },
  "node3": {
    "parent": "node1",
    "data": "Some data for node3"
  },
  "node4": {
    "parent": "node3",
    "data": "Some data for node4"
  },
  "node5": {
    "parent": "node3",
    "data": "Some data for node5"
  },
  "node36": {
    "data": "Some data for node6",
    "parent": "node1"
  },
  "node7": {
    "data": "Some data for node7",
    "parent": "node36"
  }
}
ProxyNode(node7: {'data': 'Some data for node7', 'parent': 'node36'})


In [9]:
#child7.new_data = "Some new data for node7"
#print(child7)
#del child7.new_data
#print(child7)

for k, v in child7.items():
    print(k)

child7["new_data"] = "Some new data for node7"    
print(child7)
print(tree)

del child7["new_data"]
print(child7)

print(tree.get_root().children())

data
parent
ProxyNode(node7: {'data': 'Some data for node7', 'parent': 'node36', 'new_data': 'Some new data for node7'})
{'node1': {'data': 'Some data for node1'}, 'node2': {'data': 'Some data for node2'}, 'node3': {'parent': 'node1', 'data': 'Some data for node3'}, 'node4': {'parent': 'node3', 'data': 'Some data for node4'}, 'node5': {'parent': 'node3', 'data': 'Some data for node5'}, 'node36': {'data': 'Some data for node6', 'parent': 'node1'}, 'node7': {'data': 'Some data for node7', 'parent': 'node36', 'new_data': 'Some new data for node7'}}
ProxyNode(node7: {'data': 'Some data for node7', 'parent': 'node36'})
[ProxyNode(node1: {'data': 'Some data for node1'}), ProxyNode(node2: {'data': 'Some data for node2'})]


In [10]:
print(tree)

{'node1': {'data': 'Some data for node1'}, 'node2': {'data': 'Some data for node2'}, 'node3': {'parent': 'node1', 'data': 'Some data for node3'}, 'node4': {'parent': 'node3', 'data': 'Some data for node4'}, 'node5': {'parent': 'node3', 'data': 'Some data for node5'}, 'node36': {'data': 'Some data for node6', 'parent': 'node1'}, 'node7': {'data': 'Some data for node7', 'parent': 'node36'}}


Let's modify the payload of child7.

In [11]:
#child7.parent
#print(child7.data)
print(json.dumps(tree, indent=2))

{
  "node1": {
    "data": "Some data for node1"
  },
  "node2": {
    "data": "Some data for node2"
  },
  "node3": {
    "parent": "node1",
    "data": "Some data for node3"
  },
  "node4": {
    "parent": "node3",
    "data": "Some data for node4"
  },
  "node5": {
    "parent": "node3",
    "data": "Some data for node5"
  },
  "node36": {
    "data": "Some data for node6",
    "parent": "node1"
  },
  "node7": {
    "data": "Some data for node7",
    "parent": "node36"
  }
}


In [12]:
print(tree.get_root().children())

[ProxyNode(node1: {'data': 'Some data for node1'}), ProxyNode(node2: {'data': 'Some data for node2'})]


In [13]:
child_of_log_root = tree.get_root().add_child(key="node0", data="Some data for node0")
print(child_of_log_root)

ProxyNode(node0: {'data': 'Some data for node0', 'parent': None})


In [14]:
child_of_log_root['other'] = "Some other data for node0"
print(child_of_log_root)

ProxyNode(node0: {'data': 'Some data for node0', 'parent': None, 'other': 'Some other data for node0'})


In [15]:
child_of_log_root['parent'] = 'node1'
print(json.dumps(tree, indent=2))

{
  "node1": {
    "data": "Some data for node1"
  },
  "node2": {
    "data": "Some data for node2"
  },
  "node3": {
    "parent": "node1",
    "data": "Some data for node3"
  },
  "node4": {
    "parent": "node3",
    "data": "Some data for node4"
  },
  "node5": {
    "parent": "node3",
    "data": "Some data for node5"
  },
  "node36": {
    "data": "Some data for node6",
    "parent": "node1"
  },
  "node7": {
    "data": "Some data for node7",
    "parent": "node36"
  },
  "node0": {
    "data": "Some data for node0",
    "parent": "node1",
    "other": "Some other data for node0"
  }
}


In [16]:
"node1" in tree

True

In [17]:
root_node = tree.get_root()
print(root_node)

try:
    root_node['data'] = "Some new data for root node"
except TypeError as e:
    print(e)

try:
    root_node['parent'] = None
except TypeError as e:
    print(e)

try:
    root_node.clear()
except TypeError as e:
    print(e)


print(child7)
child7.clear()

print(tree)

ProxyNode(key=__logical_root__)
__logical_root__ is immutable
__logical_root__ is immutable
ProxyNode(node7: {'data': 'Some data for node7', 'parent': 'node36'})
{'node1': {'data': 'Some data for node1'}, 'node2': {'data': 'Some data for node2'}, 'node3': {'parent': 'node1', 'data': 'Some data for node3'}, 'node4': {'parent': 'node3', 'data': 'Some data for node4'}, 'node5': {'parent': 'node3', 'data': 'Some data for node5'}, 'node36': {'data': 'Some data for node6', 'parent': 'node1'}, 'node7': {}, 'node0': {'data': 'Some data for node0', 'parent': 'node1', 'other': 'Some other data for node0'}}


In [18]:
tree.get_root().add_child(whatever=3).add_child(whatever=4).add_child(whatever=5)
print(json.dumps(tree, indent=2))

{
  "node1": {
    "data": "Some data for node1"
  },
  "node2": {
    "data": "Some data for node2"
  },
  "node3": {
    "parent": "node1",
    "data": "Some data for node3"
  },
  "node4": {
    "parent": "node3",
    "data": "Some data for node4"
  },
  "node5": {
    "parent": "node3",
    "data": "Some data for node5"
  },
  "node36": {
    "data": "Some data for node6",
    "parent": "node1"
  },
  "node7": {},
  "node0": {
    "data": "Some data for node0",
    "parent": "node1",
    "other": "Some other data for node0"
  },
  "157591a2-cf12-4881-aca3-44f50f6b8aca": {
    "whatever": 3,
    "parent": null
  },
  "c8609c5e-b147-43d8-bd51-aad2a686642b": {
    "whatever": 4,
    "parent": "157591a2-cf12-4881-aca3-44f50f6b8aca"
  },
  "81fead5f-997c-48a5-85c6-d0917ae1b593": {
    "whatever": 5,
    "parent": "c8609c5e-b147-43d8-bd51-aad2a686642b"
  }
}


In [19]:
simple_tree = tk.FlatTree({
    "root": {
        "data": "Some data for root",
#        "parent": None
    },
    "child1": {
        "data": "Some data for child1",
        "parent": "root"
    },
    "child2": {
        "data": "Some data for child2",
        "parent": "root"
    },
    "child3": {
        "data": "Some data for child3",
        "parent": "child1"
    }
})
print(json.dumps(simple_tree, indent=2))  

{
  "root": {
    "data": "Some data for root"
  },
  "child1": {
    "data": "Some data for child1",
    "parent": "root"
  },
  "child2": {
    "data": "Some data for child2",
    "parent": "root"
  },
  "child3": {
    "data": "Some data for child3",
    "parent": "child1"
  }
}


In [28]:
print(simple_tree.get_root())

print(simple_tree.get_root().get_parent())
print(simple_tree.get_node("child3").get_parent())

ProxyNode(root: {'data': 'Some data for root'})
None
ProxyNode(child1: {'data': 'Some data for child1', 'parent': 'root'})


In [21]:
import treekit.tree_converter as tc
new_tree = tc.TreeConverter.to_treenode(simple_tree)

print(json.dumps(new_tree, indent=2))

{
  "children": [
    {
      "children": [
        {
          "children": [],
          "__name__": "child3",
          "data": "Some data for child3",
          "parent": "child1"
        }
      ],
      "__name__": "child1",
      "data": "Some data for child1",
      "parent": "root"
    },
    {
      "children": [],
      "__name__": "child2",
      "data": "Some data for child2",
      "parent": "root"
    }
  ],
  "__name__": "root",
  "data": "Some data for root"
}


In [33]:
testtree = tk.FlatTree()
testtree.get_root()

treen = tk.TreeNode({
    "data": "Some data for root",
    "children": [
        {
            "data": "Some data for child1",
            "children": [
                {
                    "data": "Some data for child3",
                    "children": []
                }
            ]
        },
        {
            "data": "Some data for child2",
            "children": []
        }
    ]
    })
#print(json.dumps(treen, indent=2))

treen.name()

child_of_treen = treen.children()[0]
print(child_of_treen)
print(child_of_treen.get_parent())


{'data': 'Some data for child1', 'children': [{'data': 'Some data for child3', 'children': []}]}


In [27]:
new_flat_tree = tc.TreeConverter.to_flattree(new_tree)

TypeError: 'NoneType' object is not callable